In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [87]:
def score_ohe(data,feature,ohe):
    df = data.copy()
    for f in feature:
        df.rename(columns={f'{f}':f'{f}_{ohe}'}, inplace=True)
    return df

def make_rank(data, feature):
    df = data.copy()
    for f in feature:
        df[f"{f}_rank"] = df[f].apply(lambda x: sorted(math[f].values, reverse=True).index(x)+1)
    return df

def make_rank_bin(data, feature):
    df = data.copy()
    def make_bin(x):
        if x<=4: return 1
        elif x<=11: return 2
        elif x<=23: return 3
        elif x<=40: return 4
        elif x<=60: return 5
        elif x<=77: return 6
        elif x<=89: return 7
        elif x<=96: return 8
        else: return 9
    
    for f in feature:
        df[f"{f}_bin"] = df[f].apply(lambda x: x/len(df)*100)
        
    for f in feature:
        df[f"{f}_bin"] = df[f"{f}_bin"].apply(lambda x: make_bin(x))
    return df

def make_scaling(data,feature):
    df = data.copy()  
    scaler = MinMaxScaler()
    
    for f in feature:
        df[f"{f[:2]}_scale"] = scaler.fit_transform(df[[f]]).reshape(-1,1)
    
    return df

In [100]:
### 데이터 불러오기
math = pd.read_csv("../../1.raw_data/student-mat.csv")
port = pd.read_csv("../../1.raw_data/student-por.csv")

math.head(3)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10


In [101]:
### 데이터 스케일링
math = make_scaling(math, ["G1","G2","G3"])
port = make_scaling(port, ["G1","G2","G3"])

math.iloc[:,-6:].head(3)

,G1,G2,G3,G1_scale,G2_scale,G3_scale
0,5,6,6,0.125,0.315789,0.3
1,5,5,6,0.125,0.263158,0.3
2,7,8,10,0.250,0.421053,0.5


In [105]:
### 총합 데이터
math["GT"] = math["G1"]+math["G2"]+math["G3"]
math["GT_scale"]=math["G1_scale"]+math["G2_scale"]+math["G3_scale"]
port["GT"] = port["G1"]+port["G2"]+port["G3"]
port["GT_scale"]=port["G1_scale"]+port["G2_scale"]+port["G3_scale"]

### 총합 스케일 -> 스케일
scaler = MinMaxScaler()
math["GT_scale"] = scaler.fit_transform(math[["GT_scale"]])
port["GT_scale"] = scaler.fit_transform(port[["GT_scale"]])

math.iloc[:,-8:].head(3)

,G1,G2,G3,G1_scale,G2_scale,G3_scale,GT,GT_scale
0,5,6,6,0.125,0.315789,0.3,17,0.230907
1,5,5,6,0.125,0.263158,0.3,16,0.212990
2,7,8,10,0.250,0.421053,0.5,25,0.377380


In [58]:
math = make_rank(math, ["G1_math","G2_math","G3_math","GT_math"])

In [62]:
math = make_rank_bin(math, ["G1_math_rank","G2_math_rank","G3_math_rank","GT_math_rank"])

In [84]:
make_scaling(math, ["G1_math","G2_math","G3_math","GT_math"])

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,G3_math_rank,GT_math_rank,G1_math_rank_bin,G2_math_rank_bin,G3_math_rank_bin,GT_math_rank_bin,G1_scale,G2_scale,G3_scale,GT_scale
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,335,353,9,8,7,8,0.1250,0.315789,0.30,0.240741
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,335,358,9,8,7,8,0.1250,0.263158,0.30,0.222222
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,210,293,7,6,5,6,0.2500,0.421053,0.50,0.388889
3,GP,F,15,U,GT3,T,4,2,health,services,...,41,57,2,3,2,3,0.7500,0.736842,0.75,0.740741
4,GP,F,16,U,GT3,T,3,3,other,other,...,210,287,8,5,5,6,0.1875,0.526316,0.50,0.407407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,266,264,6,6,6,6,0.3750,0.473684,0.45,0.425926
391,MS,M,17,U,LE3,T,3,1,services,services,...,25,35,3,2,2,2,0.6875,0.842105,0.80,0.777778
392,MS,M,21,R,GT3,T,1,1,other,other,...,326,293,5,6,7,6,0.4375,0.421053,0.35,0.388889
393,MS,M,18,R,LE3,T,3,2,services,other,...,210,180,5,4,5,5,0.5000,0.631579,0.50,0.537037
